In [1]:
import os
import numpy as np
from pathlib import Path

import torch
from torch.utils.data import DataLoader

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cuda:0


In [2]:
import sys; sys.path.append("..")
from src.datasets.fcgr import FCGRDataset

In [4]:
fcgr_dataset = FCGRDataset(fcgr_dir="../data/fcgr/6mer")

In [5]:
train_dataloader = DataLoader(fcgr_dataset, batch_size=32, shuffle=True)

In [6]:
t_fcgr, t_label = next(iter(train_dataloader))

### Convolutional Neural network 


In [7]:
from src.models.cnn import CNNFCGR

model = CNNFCGR(num_classes=2, kmer=6).to(torch.float32)
# model = CNNFCGR(num_classes=2)
model.to(device)
print(model)

CNNFCGR(
  (cnn1): Conv2d(1, 24, kernel_size=(2, 2), stride=(2, 2), bias=False)
  (cnn2): Conv2d(24, 20, kernel_size=(2, 2), stride=(2, 2), bias=False)
  (cnn3): Conv2d(20, 16, kernel_size=(2, 2), stride=(2, 2), bias=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (dense): Linear(in_features=1024, out_features=2, bias=True)
)


In [8]:
# import torch

# class CNNFCGR6(torch.nn.Module):
#     def __init__(self, num_classes):
#         super().__init__()
#         self.cnn1 = torch.nn.Conv2d(in_channels=1, out_channels=4*6, kernel_size=2, stride=2, padding=0, bias=False)
#         self.cnn2 = torch.nn.Conv2d(in_channels=4*6, out_channels=4*5, kernel_size=2, stride=2, padding=0, bias=False)
#         self.cnn3 = torch.nn.Conv2d(in_channels=4*5, out_channels=4*4, kernel_size=2, stride=2, padding=0, bias=False)
#         self.flatten = torch.nn.Flatten()
#         self.dense = torch.nn.Linear(in_features=4*4*64, out_features=num_classes)

#     def forward(self, x):
#         x = self.cnn1(x)
#         x = self.cnn2(x)
#         x = self.cnn3(x)
#         x = self.flatten(x)
#         x = self.dense(x)
#         return x

In [28]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(10):
    lloss = []
    for data in train_dataloader:
        X, y = data
        X, y = X.to(device), y.to(device)
        out = model(X)  # Perform a single forward pass.
        # out = F.softmax(out, dim=-1)
        loss = criterion(out, y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        # for name, param in model.named_parameters():
        #     print(name, param.grad)
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
        
        lloss.append(loss.cpu().detach().numpy())

    m_loss = np.array(lloss).mean()

    lloss.append(loss.cpu().detach().numpy())
    print(f'Epoch: {epoch:03d}, Loss: {m_loss:.4f}') 


Epoch: 000, Loss: 0.0000
Epoch: 001, Loss: 0.0000
Epoch: 002, Loss: 0.0000
Epoch: 003, Loss: 0.0000
Epoch: 004, Loss: 0.0000
Epoch: 005, Loss: 0.0000
Epoch: 006, Loss: 0.0000
Epoch: 007, Loss: 0.0000
Epoch: 008, Loss: 0.0000
Epoch: 009, Loss: 0.0000


In [27]:
out

tensor([[-153.3554,  150.3477],
        [-158.1566,  155.3517],
        [-167.3681,  164.4639],
        [-157.2388,  155.6577],
        [-150.2856,  146.6098],
        [-156.0771,  153.2131],
        [-151.7421,  148.1100],
        [-150.0620,  146.7478],
        [-148.3430,  144.8603],
        [ 520.5497, -499.3188],
        [ 696.8740, -667.1464],
        [-155.3554,  152.5753],
        [-151.6531,  149.1301],
        [  56.8054,  -53.7015],
        [  23.1898,  -21.6798],
        [ 688.0294, -658.2831]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [26]:
torch.nn.Softmax()(out)

tensor([[0.0000e+00, 1.0000e+00],
        [0.0000e+00, 1.0000e+00],
        [0.0000e+00, 1.0000e+00],
        [0.0000e+00, 1.0000e+00],
        [0.0000e+00, 1.0000e+00],
        [0.0000e+00, 1.0000e+00],
        [0.0000e+00, 1.0000e+00],
        [0.0000e+00, 1.0000e+00],
        [0.0000e+00, 1.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [0.0000e+00, 1.0000e+00],
        [0.0000e+00, 1.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 3.2614e-20],
        [1.0000e+00, 0.0000e+00]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [24]:
torch.nn.Softmax()(out).argmax(axis=1)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0], device='cuda:0')

In [25]:
y

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0], device='cuda:0')